## 1. Load the packages

In [1]:
# Data processing packages
import numpy as np
import pandas as pd
from collections import Counter

# Machine learning packages
from sklearn.model_selection import GridSearchCV, RepeatedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MultiLabelBinarizer, FunctionTransformer
from sklearn.pipeline import Pipeline
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import SequentialFeatureSelector, RFE, SelectPercentile, chi2, mutual_info_regression, SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
import torch

# Visualization packages
import seaborn as sns
import matplotlib.pyplot as plt

# Others
import time

## 2. Read the data

In [2]:
X1 = torch.load('X1_ready')
Y1 = pd.read_csv("Y1.csv", header=None, names=['revenue '])

## 4. Feature Selection

### 4.0 Give the column names to the dataset

In [3]:
def renamingDataset(X):
    new_X = X.copy()
    new_df = pd.DataFrame(new_X)
    num_col_names = ['ratings', 'n_votes', 'production_year', 'runtime', 'release_year']
    cat_col_names = ['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Others',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War',
       'Western', 'studio_freq', 'is_adult']
    img_emb_names = []
    for i in range(2048):
        img_emb_names.append("img_emb_" + str(i))

    text_emb_names = []
    for i in range(768):
        text_emb_names.append("text_emb_" + str(i))

    all_col_names = num_col_names + cat_col_names + img_emb_names + text_emb_names

    new_df.columns = all_col_names

    return new_df

In [4]:
renamed_df_ready = renamingDataset(X1)

In [5]:
def divideDataset(df):
    new_df = df.copy()

    non_emb_df = new_df.iloc[:, :34]
    img_emb_df = new_df.iloc[:, 34:2082]
    text_emb_df = new_df.iloc[:, 2082:]

    return non_emb_df, img_emb_df, text_emb_df # pd.dataframe type

In [6]:
non_emb_df, img_emb_df, text_emb_df = divideDataset(renamed_df_ready)
non_emb_df

,ratings,n_votes,production_year,runtime,release_year,Action,Adventure,Animation,Biography,Comedy,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Thriller,War,Western,studio_freq,is_adult
0,0.606742,0.809200,1.111354,0.398460,0.933839,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001695,0.0
1,0.764045,-0.271776,-0.073389,0.079814,1.366959,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000565,0.0
2,0.539326,-0.256258,-1.596629,1.099479,-2.531122,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.025141,0.0
3,0.617978,-0.215474,-0.242638,0.398460,-0.798642,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.016949,0.0
4,0.337079,-0.265518,-1.258132,0.494053,-2.098002,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.025141,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,0.584270,-0.260586,0.011235,0.047950,-0.257241,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002825,0.0
3536,0.775281,-0.246429,-3.035245,-2.150702,1.691799,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001130,0.0
3537,0.730337,-0.271509,0.349733,-1.545276,-0.148961,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005650,0.0
3538,0.719101,0.046586,0.772856,0.175408,0.500719,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026836,0.0


### 4.1 DR on Embeddings

In [7]:
def DRembeddings(img_emb_df, text_emb_df, n_components=0.95):

    # scale the data to the range between 0 and 1 before using PCA
    scaler_emb = MinMaxScaler()
    # img_emb_df_scaled = scaler_emb.fit_transform(img_emb_df)
    # text_emb_df_scaled = scaler_emb.fit_transform(text_emb_df)
    emb_df_scaled = scaler_emb.fit_transform(pd.concat([img_emb_df, text_emb_df], axis=1))

    pca_emb = PCA(n_components=n_components)
    df_reduced_emb = pd.DataFrame(pca_emb.fit_transform(emb_df_scaled))

    emb_col_names = []
    for i in range(df_reduced_emb.shape[1]):
        emb_col_names.append("emb_" + str(i))

    df_reduced_emb.columns = emb_col_names

    return df_reduced_emb

In [8]:
df_reduced_emb = DRembeddings(img_emb_df, text_emb_df, n_components=0.9)
df_reduced_emb

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_189,emb_190,emb_191,emb_192,emb_193,emb_194,emb_195,emb_196,emb_197,emb_198
0,1.054328,-2.153237,-0.802997,-0.248707,1.349493,-0.057840,1.314132,-0.675291,1.171299,0.436517,...,-0.020153,-0.015117,0.237029,-0.028050,-0.023873,0.079219,-0.031364,-0.071154,-0.091889,-0.235259
1,2.129040,-1.483597,0.043256,0.420362,0.764355,-0.197161,-2.482356,0.903683,-1.484753,-0.512926,...,-0.194736,0.081447,0.099483,-0.115929,0.107830,0.129517,-0.144803,-0.000834,-0.156755,0.159004
2,-0.875198,1.225310,-0.543729,-1.675668,-0.747023,-0.841552,0.549715,-1.524902,-0.489789,0.147765,...,0.092746,0.026760,0.241836,0.224205,0.007772,0.136998,0.067925,-0.050262,0.116298,0.057964
3,-2.737351,-0.962277,-0.147203,0.202019,0.383081,0.646871,1.802819,0.555741,1.016410,0.509889,...,0.222160,-0.061716,0.269019,-0.029332,0.159303,0.049171,0.240762,-0.297771,-0.147129,0.116596
4,-0.188856,-1.475641,1.705273,0.440138,0.412238,-1.173195,-0.601143,-0.155727,-0.484766,-0.416415,...,-0.053639,-0.288431,0.096173,-0.151598,0.179133,0.025855,0.221040,0.155626,-0.151701,-0.016146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,2.200888,-1.601251,-1.428959,-1.419945,-0.136471,-0.784107,0.130939,-0.890358,0.586813,-0.254475,...,0.090543,-0.110694,0.100748,0.411141,-0.191849,0.070196,-0.251843,0.062231,0.173676,-0.353208
3536,2.292233,0.514620,-2.268296,1.002200,-0.883624,-0.801217,-0.707837,0.342407,-0.305779,-0.881682,...,-0.031045,0.055816,0.341487,-0.143810,0.275008,0.035487,-0.250839,-0.502362,-0.521410,0.010129
3537,-0.875170,0.782112,-1.783827,-1.313507,0.580249,-0.994133,-0.276110,-0.226078,0.496469,0.417320,...,-0.156029,0.231721,-0.045153,0.440696,-0.110274,0.025682,-0.034410,-0.120474,0.086602,-0.152570
3538,-2.001723,-1.145958,1.331780,-1.073421,0.935715,-1.775468,-0.476440,1.169051,2.360350,0.496899,...,0.156725,-0.109072,0.018352,-0.131649,-0.065725,-0.041340,-0.030564,-0.380415,0.054779,-0.108390


### 4.2 Combine non_emb with reduced_emb

In [9]:
def combineNonReducedEmb(non_em_df, reduced_emb_df):
    DR_df = pd.concat([non_em_df, reduced_emb_df], axis=1)
    return DR_df

In [10]:
DR_df = combineNonReducedEmb(non_emb_df, df_reduced_emb)
DR_df

,ratings,n_votes,production_year,runtime,release_year,Action,Adventure,Animation,Biography,Comedy,...,emb_189,emb_190,emb_191,emb_192,emb_193,emb_194,emb_195,emb_196,emb_197,emb_198
0,0.606742,0.809200,1.111354,0.398460,0.933839,0.0,1.0,0.0,0.0,1.0,...,-0.020153,-0.015117,0.237029,-0.028050,-0.023873,0.079219,-0.031364,-0.071154,-0.091889,-0.235259
1,0.764045,-0.271776,-0.073389,0.079814,1.366959,1.0,0.0,0.0,0.0,0.0,...,-0.194736,0.081447,0.099483,-0.115929,0.107830,0.129517,-0.144803,-0.000834,-0.156755,0.159004
2,0.539326,-0.256258,-1.596629,1.099479,-2.531122,0.0,0.0,0.0,0.0,0.0,...,0.092746,0.026760,0.241836,0.224205,0.007772,0.136998,0.067925,-0.050262,0.116298,0.057964
3,0.617978,-0.215474,-0.242638,0.398460,-0.798642,0.0,0.0,0.0,1.0,0.0,...,0.222160,-0.061716,0.269019,-0.029332,0.159303,0.049171,0.240762,-0.297771,-0.147129,0.116596
4,0.337079,-0.265518,-1.258132,0.494053,-2.098002,0.0,0.0,0.0,0.0,0.0,...,-0.053639,-0.288431,0.096173,-0.151598,0.179133,0.025855,0.221040,0.155626,-0.151701,-0.016146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,0.584270,-0.260586,0.011235,0.047950,-0.257241,0.0,0.0,0.0,0.0,1.0,...,0.090543,-0.110694,0.100748,0.411141,-0.191849,0.070196,-0.251843,0.062231,0.173676,-0.353208
3536,0.775281,-0.246429,-3.035245,-2.150702,1.691799,0.0,0.0,0.0,0.0,0.0,...,-0.031045,0.055816,0.341487,-0.143810,0.275008,0.035487,-0.250839,-0.502362,-0.521410,0.010129
3537,0.730337,-0.271509,0.349733,-1.545276,-0.148961,0.0,0.0,0.0,0.0,0.0,...,-0.156029,0.231721,-0.045153,0.440696,-0.110274,0.025682,-0.034410,-0.120474,0.086602,-0.152570
3538,0.719101,0.046586,0.772856,0.175408,0.500719,0.0,0.0,0.0,0.0,0.0,...,0.156725,-0.109072,0.018352,-0.131649,-0.065725,-0.041340,-0.030564,-0.380415,0.054779,-0.108390


### 4.3 Feature Selection

In [11]:
# feature_selection = SelectPercentile(mutual_info_regression, percentile=5)
# # feature_selection.fit(DR_df.to_numpy(), Y1.to_numpy().ravel())
# pip2 = Pipeline([
#
# ])

In [14]:
embeded_rf_selector = SelectFromModel(RandomForestRegressor(n_estimators=100, random_state=42), max_features=50)
embeded_rf_selector.fit(DR_df, Y1.to_numpy().ravel())

SelectFromModel(estimator=RandomForestRegressor(random_state=42),
                max_features=50)

## Pipeline

In [ ]:
def FeatureSelection(X, n_components=0.95):
    renamed_df_ready = renamingDataset(X)
    non_emb_df, img_emb_df, text_emb_df = divideDataset(renamed_df_ready)
    reduced_emb_df = DRembeddings(img_emb_df, text_emb_df, n_components=n_components)
    DR_df = combineNonReducedEmb(non_emb_df, reduced_emb_df)

    return DR_df

In [ ]:
feature_selection_transformer = FunctionTransformer(FeatureSelection)

In [ ]:
p2 = Pipeline([
    # ('FeatureSelection', feature_selection_transformer),
    ('model', Lasso())
])